# 【課題】腫瘍の良性／悪性を予測（分類）しよう

## データを読み込む

`breast_cancer_wisconsin_data.csv` を読み込みましょう。

In [1]:
# Pandasを使ってbreast_cancer_wisconsin_data.csvを読み込む（変更しないでください）
import pandas as pd
bc_data = pd.read_csv("breast_cancer_wisconsin_data.csv")

読み込んだデータについて確認します。

In [2]:
# 読み込んだデータが何行何列かをshapeで確認する（変更しないでください）
print(bc_data.shape)

(569, 33)


In [3]:
# 読み込んだデータの先頭5行を確認する（変更しないでください）
bc_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
# 読み込んだデータの先頭の1行だけを確認する（変更しないでください）
bc_data.iloc[0, :]

id                           842302
diagnosis                         M
radius_mean                   17.99
texture_mean                  10.38
perimeter_mean                122.8
area_mean                    1001.0
smoothness_mean              0.1184
compactness_mean             0.2776
concavity_mean               0.3001
concave points_mean          0.1471
symmetry_mean                0.2419
fractal_dimension_mean      0.07871
radius_se                     1.095
texture_se                   0.9053
perimeter_se                  8.589
area_se                       153.4
smoothness_se              0.006399
compactness_se              0.04904
concavity_se                0.05373
concave points_se           0.01587
symmetry_se                 0.03003
fractal_dimension_se       0.006193
radius_worst                  25.38
texture_worst                 17.33
perimeter_worst               184.6
area_worst                   2019.0
smoothness_worst             0.1622
compactness_worst           

このCSVファイルには30以上の列があります。主要な列のみ、以下に概要を記載します。

- id：連番
- diagnosis："B"か"M"の文字が格納されている（"B"：良性、"M"：悪性）
- radius_mean：中心から外周までの平均距離（半径）
- texture_mean：グレースケール（色の濃さ）の平均値
- perimeter_mean：外周の平均の長さ

今回は `radius_mean`（半径）と `perimeter_mean`（外周の長さ）を説明変数、`diagnosis`（良性か悪性か）を目的変数として分類を行ないます。

データの前処理や分割を行なう前に、スライスを使って、`radius_mean`列と`perimeter_mean`列を変数`X`、`diagnosis` 列のみを変数 `y` に格納してください。また、PandasでCSVを読み込んだため、データはDataFrame形式となっています。そのため、データはndarray形式に変換してください。

In [ ]:
import numpy as np

# スライスを使って必要な列を抽出し、ndarray形式に変換
# 説明変数: radius_mean（半径） と perimeter_mean（外周の長さ）
X = bc_data[['radius_mean', 'perimeter_mean']].values

# 目的変数: diagnosis（良性か悪性か）　
    # bc_dataというDataFrameから'diagnosis'という列名の列を抽出。結果はPandasのSeries形式になる
    # .valuesでPandas SeriesをNumPyのndarrayに変換する。機械学習ライブラリ（scikit-learn）はndarrayを期待するため
y = bc_data['diagnosis'].values

# 結果の確認
print("Xの形状:", X.shape)
print("Xのデータ型:", type(X))
print("yの形状:", y.shape)
print("yのデータ型:", type(y))

print("\nXの最初の5行:")
print(X[:5])
print("\nyの最初の5行:")
print(y[:5])

print("\nyのユニークな値:")
print(np.unique(y))

Xの形状: (569, 2)
Xのデータ型: <class 'numpy.ndarray'>
yの形状: (569,)
yのデータ型: <class 'numpy.ndarray'>

Xの最初の5行:
[[ 17.99 122.8 ]
 [ 20.57 132.9 ]
 [ 19.69 130.  ]
 [ 11.42  77.58]
 [ 20.29 135.1 ]]

yの最初の5行:
['M' 'M' 'M' 'M' 'M']

yのユニークな値:
['B' 'M']


## データの前処理を行なう

さて、`y` には "B" もしくは "M" という文字データが入っています。数値化した方がコンピュータは学習しやすくなるので、データの前処理の1つ「カテゴリ値の数値化」を実行しましょう。カテゴリ値の数値化をするには `LabelEncoder` というものを利用します。

[sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

※ `LabelEncoder` は、レッスン本文では登場しませんでしたので、以下のコードをそのまま実行すれば大丈夫です。このコードは、上記の公式ドキュメントの内容に沿って記述されています。**なお、このとき `y` は、NumPyのndarray形式に変換されている必要があります。**

In [8]:
# カテゴリ値の数値化
# （変更しないでください）
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(["B", "M"])    # 良性：0, 悪性：1
y = le.transform(y)

# 数値化した状態を確認してみる（変更しないでください）
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

## データを訓練データとテストデータに分ける

`X` および `y` を訓練データとテストデータに分けましょう。その際、訓練データ8割、テストデータ2割としてください。また、並び順がランダムになるよう、分割してください。

In [ ]:
# 訓練データ8割、テストデータ2割に分割する
# （以下にコードを書いてください）

# scikit-learnからデータ分割用の関数をインポート。機械学習でデータを訓練用とテスト用に分けるための標準的な関数
from sklearn.model_selection import train_test_split

# データを訓練データとテストデータに分割
# 訓練データ8割、テストデータ2割、ランダムに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # テストデータの割合を20%に設定
    random_state=42     # 再現性のためのシード値（任意の数値）
)

# 分割結果の確認
print("訓練データ:")
print("X_train の形状:", X_train.shape)
print("y_train の形状:", y_train.shape)

print("\nテストデータ:")
print("X_test の形状:", X_test.shape)
print("y_test の形状:", y_test.shape)

# 分割比率の確認
total_samples = len(X)
train_ratio = len(X_train) / total_samples
test_ratio = len(X_test) / total_samples

print(f"\n全データ数: {total_samples}")
print(f"訓練データ比率: {train_ratio:.1%} ({len(X_train)}件)")
print(f"テストデータ比率: {test_ratio:.1%} ({len(X_test)}件)")

# 各データセットの目的変数の分布を確認
print("\n訓練データの目的変数分布:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
for u, c in zip(unique_train, counts_train):
    label = "良性(B)" if u == 0 else "悪性(M)"
    print(f"{label}: {c}件")

print("\nテストデータの目的変数分布:")
unique_test, counts_test = np.unique(y_test, return_counts=True)
for u, c in zip(unique_test, counts_test):
    label = "良性(B)" if u == 0 else "悪性(M)"
    print(f"{label}: {c}件")

訓練データ:
X_train の形状: (455, 2)
y_train の形状: (455,)

テストデータ:
X_test の形状: (114, 2)
y_test の形状: (114,)

全データ数: 569
訓練データ比率: 80.0% (455件)
テストデータ比率: 20.0% (114件)

訓練データの目的変数分布:
良性(B): 286件
悪性(M): 169件

テストデータの目的変数分布:
良性(B): 71件
悪性(M): 43件


## モデルを作成して訓練する

ここでは、レッスン本文でも利用した「サポートベクターマシン」（`SVC`）の線形分類モデルを使います。また、作成したモデルに訓練データを渡して、学習を行ないます。

In [ ]:
# scikit-learnライブラリからSVC（Support Vector Classifier）クラスをインポート
from sklearn.svm import SVC

# SVCの線形分類モデルを作成する
model = SVC(kernel='linear', random_state=42)

# 訓練データを使って訓練を行なう
model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [11]:
# 訓練が完了したことを確認
print("モデルの訓練が完了しました")
print("使用したカーネル:", model.kernel)
print("訓練データ数:", len(X_train))

# モデルのパラメータ確認（オプション）
print("\nモデルの詳細:")
print("サポートベクター数:", model.n_support_)
print("クラス:", model.classes_)

モデルの訓練が完了しました
使用したカーネル: linear
訓練データ数: 455

モデルの詳細:
サポートベクター数: [59 59]
クラス: [0 1]


## 期待する性能が出たかを評価する

分類を行ないましょう。

In [ ]:
# テストデータを使って分類を実行する
y_pred = model.predict(X_test)

# 予測結果の確認
print("予測結果の最初の10件:")
print("実際の値:", y_test[:10])
print("予測値  :", y_pred[:10])

# 予測結果の分布を確認
unique_pred, counts_pred = np.unique(y_pred, return_counts=True)
print("\n予測結果の分布:")
for u, c in zip(unique_pred, counts_pred):
    label = "良性(B)" if u == 0 else "悪性(M)"
    print(f"{label}: {c}件")


予測結果の最初の10件:
実際の値: [0 1 1 0 0 1 1 1 0 0]
予測値  : [0 1 1 0 0 1 1 1 0 0]

予測結果の分布:
良性(B): 78件
悪性(M): 36件


この分類結果を用いて、モデルの評価を行ないます。

まずは、予測値と実際の値をそのまま表示してください。

In [14]:
# 予測値を表示する

print("予測値:")
print(y_pred)

予測値:
[0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1
 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1
 0 0 1]


In [15]:
# 実際の値を表示する

print("\n実際の値:")
print(y_test)




実際の値:
[0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1
 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 0 1 0 0 1
 0 0 1]


予測の精度を混同行列で示しましょう。

In [16]:
# 混同行列で集計結果を表示する

# より見やすい形で比較表示
print("\n予測値と実際の値の比較:")
print("インデックス | 実際の値 | 予測値 | 一致")
print("-" * 35)
for i in range(len(y_test)):
    match = "○" if y_test[i] == y_pred[i] else "×"
    actual_label = "良性(B)" if y_test[i] == 0 else "悪性(M)"
    pred_label = "良性(B)" if y_pred[i] == 0 else "悪性(M)"
    print(f"{i:8d} | {actual_label:7s} | {pred_label:7s} | {match}")

# 全体の件数確認
print(f"\n全テストデータ数: {len(y_test)}件")
print(f"予測データ数: {len(y_pred)}件")

# 一致・不一致の件数
correct = np.sum(y_test == y_pred)
incorrect = np.sum(y_test != y_pred)
print(f"正解: {correct}件")
print(f"不正解: {incorrect}件")



予測値と実際の値の比較:
インデックス | 実際の値 | 予測値 | 一致
-----------------------------------
       0 | 良性(B)   | 良性(B)   | ○
       1 | 悪性(M)   | 悪性(M)   | ○
       2 | 悪性(M)   | 悪性(M)   | ○
       3 | 良性(B)   | 良性(B)   | ○
       4 | 良性(B)   | 良性(B)   | ○
       5 | 悪性(M)   | 悪性(M)   | ○
       6 | 悪性(M)   | 悪性(M)   | ○
       7 | 悪性(M)   | 悪性(M)   | ○
       8 | 良性(B)   | 良性(B)   | ○
       9 | 良性(B)   | 良性(B)   | ○
      10 | 良性(B)   | 良性(B)   | ○
      11 | 悪性(M)   | 悪性(M)   | ○
      12 | 良性(B)   | 良性(B)   | ○
      13 | 悪性(M)   | 悪性(M)   | ○
      14 | 良性(B)   | 良性(B)   | ○
      15 | 悪性(M)   | 悪性(M)   | ○
      16 | 良性(B)   | 良性(B)   | ○
      17 | 良性(B)   | 良性(B)   | ○
      18 | 良性(B)   | 良性(B)   | ○
      19 | 悪性(M)   | 悪性(M)   | ○
      20 | 悪性(M)   | 良性(B)   | ×
      21 | 良性(B)   | 良性(B)   | ○
      22 | 悪性(M)   | 悪性(M)   | ○
      23 | 良性(B)   | 良性(B)   | ○
      24 | 良性(B)   | 良性(B)   | ○
      25 | 良性(B)   | 良性(B)   | ○
      26 | 良性(B)   | 良性(B)   | ○
      27 | 良性(B)   | 良性(B)   | ○
 

最後に `classification_report()` を実行し、結果を表示してください。

In [17]:
# classification_report()の実行結果を表示する

from sklearn.metrics import classification_report

# classification_report()を実行し、結果を表示
print("分類レポート:")
print(classification_report(y_test, y_pred))


分類レポート:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94        71
           1       0.97      0.81      0.89        43

    accuracy                           0.92       114
   macro avg       0.93      0.90      0.91       114
weighted avg       0.93      0.92      0.92       114

